<a href="https://colab.research.google.com/github/Itsuki966/it_communication_kuwa/blob/main/m_league.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

In [3]:
# jsonファイルの読み込み
with open('/content/drive/MyDrive/2023-2024_paifu/L001_S002_0008_01A.json') as f:
  paifu = json.load(f)

## kyokustartとkyokuendのリスト内での順番を取得

In [4]:
kyokustart = []
kyokuend = []

for i in range(len(paifu)):
  c = paifu[i]["cmd"]
  if c == 'kyokustart':
    kyokustart.append(i)
  elif c == 'kyokuend':
    kyokuend.append(i)

In [ ]:
print(f'kyokustart:{kyokustart}, kyokuend:{kyokuend}, lenght{len(kyokustart)}')

kyokustart:[6, 95, 271, 451, 556, 684, 864, 1009, 1081, 1152, 1340, 1417, 1493], kyokuend:[94, 270, 450, 555, 683, 863, 1008, 1080, 1151, 1339, 1416, 1492, 1628], lenght13


# **ゲームスタートまでの準備の関数**

In [5]:
# ゲームスタートまでの準備
def game_ready (paifu, kyoku):

  tehai = {}
  alldict = {}

  for i in range(kyokustart[kyoku], kyokuend[kyoku]+1):
    cmd = paifu[i]['cmd']

    # 親を保存
    if cmd == "dice":
      first_player = paifu[i+2]['args'][0]

    # 配牌
    elif cmd == "haipai":
      player = paifu[i]['args'][0]
      if player not in tehai:
        tehai[player] = [paifu[i]['args'][1][l] + paifu[i]['args'][1][l+1] for l in range(0,len(paifu[i]['args'][1])-1,2)]
      else:
        if len(paifu[i]['args'][1]) != 2:
          for l in range(0, len(paifu[i]['args'][1])-1,2):
            pai = paifu[i]['args'][1][l] + paifu[i]['args'][1][l+1]
            tehai[player].append(pai)
        else:
          tehai[player].append(paifu[i]['args'][1])


  # 全てを保存する辞書の作成
  alldict['tehai'] = tehai[first_player]
  alldict['tsumo'] = [0]
  alldict['tedashi'] = [0]

  alldict['dora'] = [0,0,0,0]
  # alldict['naki'] = [0,0,0]
  # alldict['richi'] = {""}

  # 各プレイヤーごとに捨て牌のリストを作成し、辞書に追加
  key = list(tehai.keys())
  for k in key:
    alldict[k] = {"sutehai":[0*s for s in range(1,26)],"naki":[[0*i for i in range(1,5)],[0*i for i in range(1,5)],[0*i for i in range(1,5)],[0*i for i in range(1,5)]], "richi":[0]}

  return tehai, alldict


# **ゲームの処理用の関数**

In [8]:
def game(paifu, tehai, alldict, kyoku):
  naki_count = 0
  dora_count = 0
  game_data = []

  for n in range(kyokustart[kyoku], kyokuend[kyoku]+1):
    cmd = paifu[n]['cmd']

    # ツモから牌を切るまで
    if cmd == "tsumo":
      player = paifu[n]['args'][0]
      tsumo = paifu[n]['args'][2]
      alldict['tehai'] = tehai[player]
      if paifu[n+1]['cmd'] == 'sutehai':
        sutehai = paifu[n+1]['args'][1]
        tehai[player].append(tsumo)
        tehai[player].remove(sutehai)
      elif paifu[n+1]['args'][1] == 'richi':
        sutehai = paifu[n+2]['args'][1]
        tehai[player].append(tsumo)
        tehai[player].remove(sutehai)
      elif paifu[n+1]['args'][1] == 'tsumo' or paifu[n+1]['args'][1] == 'ron':
        pass
      else:
        sutehai = 0

      alldict['tsumo'][0] = tsumo
      alldict['tedashi'][0] = sutehai
      alldict[player]['sutehai'].append(sutehai)
      del alldict[player]['sutehai'][0]
      print(alldict)


    elif cmd == "dora":
      dora = paifu[n]['args'][0]
      alldict['dora'][dora_count] = dora
      dora_count += 1
      print(alldict)


    elif cmd == "agari":
      print(paifu[n]['args'])

    elif cmd == "ryukyoku":
      print("流局")

    # 鳴き
    elif cmd == "say":
      say = paifu[n]['args'][1]
      player = paifu[n]['args'][0]

      if say == 'pon':
        pon = paifu[n-1]['args'][1]
        sutehai = paifu[n+2]['args'][1]

        tehai[player].append(pon)
        tehai[player].remove(sutehai)

        naki_list = alldict[player]['naki'][naki_count]
        print(alldict)

        for n in range(0,3):
          naki_list[n] = pon

        naki_count += 1
        alldict['tsumo'][0] = pon
        alldict['tedashi'][0] = sutehai
        alldict[player]['sutehai'].append(sutehai)
        del alldict[player]['sutehai'][0]
        print(alldict)


      elif say == 'chi':
        chi = paifu[n-1]['args'][1]
        sutehai = paifu[n+2]['args'][1]
        open_pai = paifu[n+1]['args'][1].replace("<","").replace(">","")

        alldict[player]['naki'][naki_count] = [open_pai[l] + open_pai[l+1] for l in range(0,len(open_pai)-1,2)]
        alldict[player]['naki'][naki_count].append(0)

        tehai[player].append(chi)
        tehai[player].remove(sutehai)

        naki_count += 1
        alldict['tsumo'][0] = chi
        alldict['tedashi'][0] = sutehai
        alldict[player]['sutehai'].append(sutehai)
        del alldict[player]['sutehai'][0]
        print(alldict)


      elif say == 'kan':
        kan = paifu[n-1]['args'][2]

        tehai[player].append(kan)

        naki_list = alldict[player]['naki'][naki_count]
        for n in range(0,4):
          naki_list[n] = kan

        naki_count += 1
        alldict['tsumo'][0] = kan
        print(alldict)


      elif say == 'richi':
        alldict[player]['richi'][0] = 1
        print(alldict)




# **実行**

In [9]:
for l in range(len(kyokustart)):

  tehai, alldict = game_ready(paifu, l)
  input_list = game(paifu, tehai, alldict, l)

{'tehai': ['2m', '2m', '9m', '9m', '9m', '1p', '6p', '7p', '2s', '5z', '5z', '6z', '7z'], 'tsumo': [0], 'tedashi': [0], 'dora': ['1m', 0, 0, 0], 'A0': {'sutehai': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'naki': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'richi': [0]}, 'B0': {'sutehai': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'naki': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'richi': [0]}, 'C0': {'sutehai': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'naki': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'richi': [0]}, 'D0': {'sutehai': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'naki': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'richi': [0]}}
{'tehai': ['2m', '2m', '9m', '9m', '9m', '6p', '7p', '2s', '5z', '5z', '6z', '7z', '8p'], 'tsumo': ['8p'], 'tedashi': ['1p'], 'dora': ['1m', 0, 0, 0], 